# Importing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  # Use scikeras for scikit-learn compatibility
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

In [2]:
# Set option to ensure charts are displayed inline in the notebook

%matplotlib inline

In [3]:
# importing climate and pleasant weather data sets
path = r'C:\Users\Administrator\Desktop\Projects (Carrer foundry)\Data sets'
df_clean = pd.read_csv(os.path.join(path, 'Climate_Cleaned.csv'))
pleasant_clean = pd.read_csv(os.path.join(path, 'Pleasant_Day_Cleaned.csv'))
climate = pd.read_csv(os.path.join(path, 'Dataset-weather-prediction-dataset-processed.csv'))
climate_answers = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

# Data prep

In [7]:
df_clean.drop('Unnamed: 0', axis =1, inplace = True)

In [8]:
# Turn X and answers from a df to arrays

X = np.array(df_clean)
y = np.array(pleasant_clean)

In [9]:
X = X.reshape(-1,15,9)
X.shape

(22950, 15, 9)

In [10]:
y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [11]:
# Check y layout

from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'binary'

In [12]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


# Bayesian optimization

In [13]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [14]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, params={'callbacks':[es]}).mean()
    return score

In [15]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), 
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (2, 64), 
    'epochs':(10, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/16


C:\Users\Administrator\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


226/226 - 2s - 10ms/step - accuracy: 0.9958 - loss: 2.6711
Epoch 2/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 2.5839
Epoch 3/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 2.3473
Epoch 4/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 1.9227
Epoch 5/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 1.4141
Epoch 6/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.9319
Epoch 7/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.5367
Epoch 8/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.2836
Epoch 9/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.1578
Epoch 10/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.0980
Epoch 11/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.0670
Epoch 12/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.0491
Epoch 13/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.0380
Epoch 14/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.0305
Epoch 15/16
226/226 - 1s - 3ms/step - a

C:\Users\Administrator\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


226/226 - 2s - 8ms/step - accuracy: 0.9985 - loss: 2.6729
Epoch 2/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 2.5989
Epoch 3/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 2.3777
Epoch 4/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 1.9524
Epoch 5/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 1.4355
Epoch 6/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.9611
Epoch 7/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.5803
Epoch 8/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.3200
Epoch 9/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.1792
Epoch 10/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.1099
Epoch 11/16
226/226 - 1s - 2ms/step - accuracy: 1.0000 - loss: 0.0740
Epoch 12/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.0536
Epoch 13/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.0409
Epoch 14/16
226/226 - 1s - 3ms/step - accuracy: 1.0000 - loss: 0.0326
Epoch 15/16
226/226 - 1s - 2ms/step - ac

...

223/223 - 2s - 9ms/step - accuracy: 0.0000e+00 - loss: 2.7044
Epoch 2/11
223/223 - 0s - 2ms/step - accuracy: 0.0000e+00 - loss: 2.6945
Epoch 3/11
223/223 - 0s - 2ms/step - accuracy: 0.0000e+00 - loss: 2.6834
Epoch 4/11
223/223 - 0s - 2ms/step - accuracy: 0.0000e+00 - loss: 2.6710
Epoch 5/11
223/223 - 0s - 2ms/step - accuracy: 7.2622e-05 - loss: 2.6574
Epoch 6/11
223/223 - 0s - 2ms/step - accuracy: 0.0000e+00 - loss: 2.6429
Epoch 7/11
223/223 - 0s - 2ms/step - accuracy: 0.0000e+00 - loss: 2.6268
Epoch 8/11
223/223 - 0s - 2ms/step - accuracy: 0.0000e+00 - loss: 2.6093
Epoch 9/11
223/223 - 0s - 2ms/step - accuracy: 2.9049e-04 - loss: 2.5909
Epoch 10/11
223/223 - 0s - 2ms/step - accuracy: 4.3573e-04 - loss: 2.5713
Epoch 11/11
223/223 - 0s - 2ms/step - accuracy: 4.3573e-04 - loss: 2.5502
56/56 - 0s - 4ms/step
Epoch 1/11


C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\utils\_response.py", line 211, in _get_response_values
    y_pred = prediction_method(X)
             ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\Li

223/223 - 2s - 9ms/step - accuracy: 2.9049e-04 - loss: 2.6549
Epoch 2/11
223/223 - 0s - 2ms/step - accuracy: 5.0835e-04 - loss: 2.6444
Epoch 3/11
223/223 - 0s - 2ms/step - accuracy: 7.2622e-04 - loss: 2.6324
Epoch 4/11
223/223 - 0s - 2ms/step - accuracy: 0.0021 - loss: 2.6187
Epoch 5/11
223/223 - 0s - 2ms/step - accuracy: 0.0042 - loss: 2.6039
Epoch 6/11
223/223 - 0s - 2ms/step - accuracy: 0.0103 - loss: 2.5874
Epoch 7/11
223/223 - 0s - 2ms/step - accuracy: 0.0219 - loss: 2.5691
Epoch 8/11
223/223 - 0s - 2ms/step - accuracy: 0.0460 - loss: 2.5495
Epoch 9/11
223/223 - 0s - 2ms/step - accuracy: 0.0947 - loss: 2.5274
Epoch 10/11
223/223 - 0s - 2ms/step - accuracy: 0.1695 - loss: 2.5035
Epoch 11/11
223/223 - 0s - 2ms/step - accuracy: 0.2834 - loss: 2.4772
56/56 - 0s - 4ms/step
| 16        | nan       | 0.1527    | 61.54     | 0.04459   | 0.04292   | 10.56     | 2.134     | 2.865     | 2.332     | 0.8244    | 11.2      | 0.542     | 3.217     |


C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\utils\_response.py", line 211, in _get_response_values
    y_pred = prediction_method(X)
             ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\anaconda3\Li

ValueError: Input y contains NaN.

In [16]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'relu',
 'batch_size': 62,
 'dropout': 0.04458809273414399,
 'dropout_rate': 0.04291858581702072,
 'epochs': 11,
 'kernel': 2.1337168665881894,
 'layers1': 3,
 'layers2': 2,
 'learning_rate': 0.8244308937956231,
 'neurons': 11,
 'normalization': 0.5420152984797261,
 'optimizer': <keras.src.optimizers.adadelta.Adadelta at 0x2590ba39a60>}

# CNN Optimized

In [17]:
# Set the model with optimized hyperparameters

epochs = 11
batch_size = 62

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 3
layers2 = 2
activation = 'relu'
kernel = int(round(2.1337168665881894))  # Rounded kernel size for Conv1D
neurons = 11
normalization = 0.5420152984797261
dropout = 0.04458809273414399
dropout_rate = 0.04291858581702072
optimizer = Adadelta(learning_rate=0.8244308937956231)  # Instantiate RMSprop with learning rate

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model.summary()

Model: "sequential_80"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_80 (Conv1D)              │ (None, 14, 11)         │           209 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, 14, 11)         │            44 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_410 (Dense)               │ (None, 14, 11)         │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_411 (Dense)               │ (None, 14, 11)         │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_412 (Dense)               │ (None, 14, 11)         │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_413 (Dense)               │ (None, 14, 11)         │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_414 (Dense)               │ (None, 14, 11)         │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_80 (MaxPooling1D) │ (None, 7, 11)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_80 (Flatten)            │ (None, 77)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_415 (Dense)               │ (None, 15)             │         1,170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,083 (8.14 KB)

 Trainable params: 2,061 (8.05 KB)

 Non-trainable params: 22 (88.00 B)

In [19]:
# Put the y_test set back into a one-hot configuration

y_train_one_hot = to_categorical(y_train, num_classes=15)

In [20]:
# Check shapes

print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [21]:
# Compile the model with categorical_crossentropy

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [22]:
# Fit the model to the data

model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/11
278/278 - 2s - 7ms/step - accuracy: 0.9518 - loss: 0.1436
Epoch 2/11
278/278 - 0s - 2ms/step - accuracy: 1.0000 - loss: 1.5758e-05
Epoch 3/11
278/278 - 0s - 2ms/step - accuracy: 1.0000 - loss: 4.3562e-06
Epoch 4/11
278/278 - 0s - 2ms/step - accuracy: 1.0000 - loss: 2.5043e-06
Epoch 5/11
278/278 - 0s - 2ms/step - accuracy: 1.0000 - loss: 1.7607e-06
Epoch 6/11
278/278 - 0s - 2ms/step - accuracy: 1.0000 - loss: 1.3464e-06
Epoch 7/11
278/278 - 0s - 2ms/step - accuracy: 1.0000 - loss: 1.0607e-06
Epoch 8/11
278/278 - 1s - 2ms/step - accuracy: 1.0000 - loss: 8.9252e-07
Epoch 9/11
278/278 - 0s - 2ms/step - accuracy: 1.0000 - loss: 7.7783e-07
Epoch 10/11
278/278 - 0s - 2ms/step - accuracy: 1.0000 - loss: 6.7021e-07
Epoch 11/11
278/278 - 0s - 2ms/step - accuracy: 1.0000 - loss: 6.0851e-07


In [23]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [24]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [25]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [26]:
# Evaluate

print(confusion_matrix(y_test, y_pred, stations))

Pred   BASEL
True        
BASEL   5738
